In [136]:
# -- public imports

import os
import nltk
import spacy
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import keras
from keras import preprocessing
from keras.preprocessing.sequence import pad_sequences

import torch.nn as nn
import torch
import torch.optim as optim
import torch.nn.functional as F


In [158]:
data = pd.read_json("/Users/feng/downloads/data/bio/TUDarmstadt_postprocessed.json")

In [168]:
df = data.iloc[:10]

In [162]:
largest_len = max(len(s.split()) for s in list(df.text))
print(f"biggest sentence has {largest_len} words")

biggest sentence has 422 words


In [163]:
output_labels = ['O', 'B-MajorClaim', 'I-MajorClaim', 'B-Claim', 'I-Claim', 'B-Premise','I-Premise']

labels_to_ids = {v:k for k,v in enumerate(output_labels)}
ids_to_labels = {k:v for k,v in enumerate(output_labels)}

In [213]:
word_to_ids = {"UNK":0}
for sentence in list(df.text):
    for word in sentence.split():
        if word not in word_to_ids.keys():
            word_to_ids[word] = len(word_to_ids)

In [170]:
df.head()

,doc_id,text,predictionString,labels
0,essay001,"Should students be taught to compete or to cooperate?\n\nIt is always said that competition can effectively promote the development of economy. In order to survive in the competition, companies continue to improve their products and service, and as a result, the whole society prospers. However, when we discuss the issue of competition or cooperation, what we are concerned about is not the whole society, but the development of an individual's whole life. From this point of view, I firmly believe that we should attach more importance to cooperation during primary education .\nFirst of all, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, O, O, O, O, B-Claim, I-Claim, I-Claim, I-Claim, ...]"
1,essay002,"More people are migrating to other countries than ever before\n\nThe last 50 years have seen an increasing number of immigrants to other countries. People moved due to a number of reasons, namely better educations or higher salary jobs. Some people thought that they should follow the local customs in order to integrate into their adopted countries’ cultures. However I strongly believe that they are able to sustain their cultural identities and doing so help they keep their origin values .\nFirstly, maintaining one’s cultural identity is a key important rule to help individuals emerge in ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, O, O, B-Premise, I-Premise, I-Premise, I-Premise, I-Premise, I-Premise, I-Premise, I-Premise, I-Premise, I-Premise, I-Premise, I-Premise, I-Premise, I-Premise, I-Premis..."
2,essay003,"International tourism is now more common than ever before\n\nThe last decade has seen an increasing number of tourists traveling to visit natural wonder sights, ancient heritages and different cultures around the world. While some people might think that this international tourism has negative effects on the destination countries, I would contend that it has contributed to the economic development as well as preserved the culture and environment of the tourist destinations . \nFirstly, international tourism promotes many aspects of the destination country’s economy in order to serve vari...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 7

In [215]:
def convert_labelId(str_list):
    l = []
    for s in str_list:
        l.append(labels_to_ids[s])
    return l
df['label_idx'] = df['labels'].apply(convert_labelId)

In [216]:
def convert_wId(str_list):
    sentence = str_list.split()
    wid = []
    for word in sentence:
        wid.append(word_to_ids[word])
    return wid
df['word_idx'] = df.text.apply(convert_wId)
df.head()

,doc_id,text,predictionString,labels,label_idx,word_idx
0,essay001,"Should students be taught to compete or to cooperate?\n\nIt is always said that competition can effectively promote the development of economy. In order to survive in the competition, companies continue to improve their products and service, and as a result, the whole society prospers. However, when we discuss the issue of competition or cooperation, what we are concerned about is not the whole society, but the development of an individual's whole life. From this point of view, I firmly believe that we should attach more importance to cooperation during primary education .\nFirst of all, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, O, O, O, O, B-Claim, I-Claim, I-Claim, I-Claim, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 3, 4, 4, 4, ...]","[1, 2, 3, 4, 5, 6, 7, 5, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 5, 24, 25, 18, 26, 27, 28, 5, 29, 30, 31, 32, 33, 32, 34, 35, 36, 18, 37, 38, 39, 40, 41, 42, 43, 18, 44, 20, 14, 7, 45, 46, 42, 47, 48, 49, 10, 50, 18, 37, 51, 52, 18, 19, 20, 53, 54, 37, 55, 56, 57, 58, 20, 59, 60, 61, 62, 13, 42, 63, 64, 65, 66, 5, 67, 68, 69, 70, 71, 72, 20, 73, 74, 45, 75, 15, ...]"
1,essay002,"More people are migrating to other countries than ever before\n\nThe last 50 years have seen an increasing number of immigrants to other countries. People moved due to a number of reasons, namely better educations or higher salary jobs. Some people thought that they should follow the local customs in order to integrate into their adopted countries’ cultures. However I strongly believe that they are able to sustain their cultural identities and doing so help they keep their origin values .\nFirstly, maintaining one’s cultural identity is a key important rule to help individuals emerge in ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, I-MajorClaim, O, O, B-Premise, I-Premise, I-Premise, I-Premise, I-Premise, I-Premise, I-Premise, I-Premise, I-Premise, I-Premise, I-Premise, I-Premise, I-Premise, I-Premise, I-Premis...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [219]:
sentences = list(df.word_idx)
max_len = 512 #??
X = [[word for word in sentence] for sentence in sentences]

In [221]:
new_X = []
for seq in X:
    new_seq = []
    for i in range(max_len):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append(0)
    new_X.append(new_seq)

In [222]:
labels = list(df.label_idx)
y = pad_sequences(maxlen=max_len,sequences=labels,padding='post',value=0)

In [223]:
X_tr,X_te,y_tr,y_te = train_test_split(new_X,y,test_size=0.1,random_state=2)

In [224]:
def generate_batch(x,y,batch_size=4):
    length = len(x)
    words = []
    labels= []
    
    for i in range(length):
        words.append(x[i])
        labels.append(y[i])
        
        if len(words)==batch_size:
            yield words,labels
            words = []
            labels = []
    

In [242]:
class LSTM_NER(nn.Module):
    def __init__(self,embedding_dim,hidden_dim,vocab_size,target_size):
        super(LSTM_NER,self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embedding = nn.Embedding(vocab_size,embedding_dim)
        self.lstm = nn.LSTM(embedding_dim,hidden_dim,batch_first=True)
        self.hidden2tag = nn.Linear(hidden_dim,target_size)
    def forward(self,sentence):
        
        embeds = self.word_embedding(sentence) #??
        lstm_out,_ = self.lstm(embeds)
        score = F.softmax(self.hidden2tag(lstm_out),dim=-1)
        return score
    

In [243]:
EMBEDDING_DIM = 300 #??
HIDDEN_DIM = 64
model = LSTM_NER(EMBEDDING_DIM,HIDDEN_DIM,len(word_to_ids.keys()),len(labels_to_ids.keys()))
loss_function = nn.CrossEntropyLoss(ignore_index=-1,reduction='mean')
optimizer = optim.SGD(model.parameters(),lr=0.1)

In [278]:
#Training
acc_list = []
loss_list = []
epochs = 3
running_loss = 0

for epoch in range(epochs):
    print(f"Epoch {epoch}")
    acc = 0
    loss = 0
    i = 0
    for idx,(sentences,labels) in enumerate(generate_batch(X_tr,y_tr)):
        optimizer.zero_grad()
        sentence = torch.tensor(sentences,dtype=torch.long)
        label = torch.tensor(labels,dtype=torch.long)
        tag_scores = model(sentence)
        loss = loss_function(tag_scores.view(-1,7),label.view(-1))
        loss.backward()
        optimizer.step()
        
        argmaxed = torch.argmax(tag_scores,dim=-1)
        mask = label>-1
        relevant = argmaxed[mask]
        acc = ((relevant==label[mask]).sum()/relevant.shape[0]).item()
        loss = loss.item()
        acc_list.append(acc)
        loss_list.append(loss)
        running_loss = 0.99*running_loss+0.01*loss
        

Epoch 0


<ipython-input-278-580fe5168114>:15: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  label = torch.tensor(labels,dtype=torch.long)


Epoch 1
Epoch 2
